# Use Arucos to create boreholes and cross-sections 

In [ ]:
import os
#Run this cell only if you want to run gempy in cuda mode (requires cuda and a dedicated cpu).
#Initial compilation will take a bit longer but the model will run much faster 
os.environ["THEANO_FLAGS"] = "mode=FAST_RUN,device=cuda"

In [ ]:
#import gempy
import gempy as gp

## Initialize Sensor and Projector

In [ ]:
#save the paths to the calibration files 
from sandbox import _calibration_dir
_calibprojector = _calibration_dir + "my_projector_calibration.json"
_calibsensor = _calibration_dir + "my_sensor_calibration.json"

In [ ]:
#Import Sensor
from sandbox.sensor import Sensor
sensor = Sensor(calibsensor=_calibsensor, name="kinect_v2")

In [ ]:
# Import projector
from sandbox.projector import Projector
projector = Projector(calibprojector=_calibprojector)


## Initialize marker detection

In [ ]:
#Initialize the aruco detection
from sandbox.markers import MarkerDetection
aruco = MarkerDetection(sensor=sensor)

## Initialize main thread for live update

In [ ]:
from sandbox.main_thread import MainThread
main = MainThread(sensor=sensor, projector=projector, aruco=aruco)

In [ ]:
# Start the thread 
main.run()

In [ ]:
main.widget_plot_module()

## Initialize the gempy module 

In [ ]:
main.stop()
from sandbox.modules.gempy import GemPyModule
gpsb= GemPyModule(geo_model = None, 
                  extent = sensor.extent, 
                  box=sensor.physical_dimensions, 
                  load_examples = True,
                  name_example = ['Horizontal_layers'])


## Add the module to the main thread

In [ ]:
main.add_module(name='gempy', module=gpsb)
main.run()

In [ ]:
#Turn on Hill Shading
gpsb.show_hillshades = True
gpsb.show_lith = True
gpsb.show_boundary = True
gpsb.show_contour = False
gpsb.show_only_faults = False
gpsb.show_fill_contour = False


## Add cross section

In [ ]:
# Set section dictionary
gpsb.set_section_dict((10,10), (800,800), "Section1") 

In [ ]:
# show section traces
gpsb.show_section_traces() 

In [ ]:
# Possible cross sections to show
gpsb.model_sections_dict.keys()

In [ ]:
# If no filling run again
_ = gpsb.show_cross_section("Section1")

## Cross section with aruco markers

There can only be 2 aruco markers displayed in the model

In [ ]:
if len(gpsb.modelspace_arucos) == 2 and 'Aruco_section' in gpsb.model_sections_dict.keys():
    _ = gpsb.show_cross_section("Aruco_section")
else:
    print('Only 2 aruco markers can be present to work')

## Show boreholes
This still gives some trouble when faults are in the model

In [ ]:
# Place synthetic boreholes
main.stop()
gpsb.set_borehole_dict((10, 20), "borehole1")
gpsb.set_borehole_dict((200, 500), "borehole2")
gpsb.set_borehole_dict((500, 500), "borehole3")
gpsb.set_borehole_dict((900, 500), "borehole4")
gpsb.set_borehole_dict((100, 100), "borehole5")
gpsb.set_borehole_dict((600, 700), "borehole6")
gpsb.set_borehole_dict((200, 150), "borehole7")
gpsb.set_borehole_dict((150, 200), "borehole8")
gpsb._get_polygon_data()
if len(gpsb.borehole_tube) > 0:
    p = gpsb.plot_boreholes(notebook=False, background=False)
    p.show()
main.run()

## Show boreholes with arucos

In [ ]:
#Place as many aruco markers as you want in the model
lis = list(gpsb.borehole_dict.keys())
for name in lis:
    if 'aruco' not in name:
        gpsb.remove_borehole_dict(name)
main.stop()
gpsb._get_polygon_data()
if len(gpsb.borehole_tube) > 0:
    p = gpsb.plot_boreholes(notebook=False, background=False)
    p.show()
else:
    print('No aruco boreholes to show. Make sure the arucos are detected')

main.run()

# Other additional nice features

In [ ]:
# Pyvista 3d model
main.stop()
gpsb.plot_3d_model()
main.run()